# Logistic ~ Week 5
<hr>

## * Kelebihan:
 1. Accuracy cukup besar 0.99966
 2. Algoritma Mudah Dipahami
## * Kekurangan:
 1. Lemot ~ 15  - 20 menit runtime.
<p style="font-size : 10pt">Note : Program yang lebih cepet & lebih akurat sama Dimas</p>

## Import Libraries
Import dulu gan ~

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import re
from tqdm import tqdm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Loading Dataset
Load datanya~

In [4]:
data = pd.read_csv('delivery_orders_march.csv')

## Checking NAN Value
Menghitung banyaknya NAN per kolom.

In [5]:
data.isna().sum()

orderid                      0
pick                         0
1st_deliver_attempt          0
2nd_deliver_attempt    1819311
buyeraddress                 0
selleraddress                0
dtype: int64

## Membuat Dictionary Expected Hari Pengiriman
Membuat dictionary yang berisi kota asal, kota tujuan, dan ketentuan waktu pengiriman sesuai data yang di excel.

In [16]:
city = list(map(lambda x : x.lower(), ['Metro Manila', 'Luzon', 'Visayas', 'Mindanao'])) # List kota yang ada
direc, k, target  = {}, 0, [3, 5, 7, 7, 5, 7, 7, 7, 7, 7]
for i in range(len(city)):
  direc[f'{city[i]}'] = {}
  for j in range(i, len(city)):
    direc[f'{city[i]}'][f'{city[j]}'] = target[k]
    k += 1
print(direc)

{'metro manila': {'metro manila': 3, 'luzon': 5, 'visayas': 7, 'mindanao': 7}, 'luzon': {'luzon': 5, 'visayas': 7, 'mindanao': 7}, 'visayas': {'visayas': 7, 'mindanao': 7}, 'mindanao': {'mindanao': 7}}


## Mengubah Format Timestamp Menjadi Datetime
Mengubah format timestamp menjadi datetime sesuai dengan ketentuan (`Tahun-Bulan-Tanggal` GMT +8).

```
GMT +8 Datetime format = to_datetime(Timestamp + 8 * 3600)
```

In [7]:
data['2nd_deliver_attempt'] = data['2nd_deliver_attempt'].fillna(0)   # Mengganti baris yang kosong pada kolom '2nd_deliver_attempt'2nd_deliver_attempt'
data['pick'] = pd.to_datetime(data['pick'].apply(lambda x : datetime.fromtimestamp(x + 8 * 3600).strftime('%Y-%m-%d')))
data['1st_deliver_attempt'] = pd.to_datetime(data['1st_deliver_attempt'].apply(lambda x : datetime.fromtimestamp(x + 8 * 3600).strftime('%Y-%m-%d')))
data['2nd_deliver_attempt'] = pd.to_datetime(data['2nd_deliver_attempt'].apply(lambda x : datetime.fromtimestamp(x + 8 * 3600).strftime('%Y-%m-%d')))
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,1970-01-01,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04,2020-03-06,1970-01-01,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05,2020-03-07,1970-01-01,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04,2020-03-06,1970-01-01,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


## Memprediksi Telat Tidaknya Pengiriman
Main Idea:
### 1. Menghitung Expected Day
* Mencari Kota Asal dan Kota Tujuan pada setiap pengiriman.
* Memeriksa waktu pengiriman berdasarkan table Expected yang sudah dibuat.

### 2. Menghitung Banyak Hari Kerja
* Menghitung banyak hari kerja pengiriman dengan mengabaikan hari libur menggunakan fungsi `busday_count` dari `numpy`.

### 3. Membandingkan Waktu Pengiriman dengan Expected Day
* Jika Expected > Waktu pengiriman maka tidak telat `0`.
* Sebaliknya jika Expected < Waktu pengiriman maka telat `1`.

In [10]:
def City(string):
  """
  Fungsi untuk mencari nama kota pada alamat.
  """
  city_re = re.compile('|'.join(city))
  kota = city_re.findall(string.lower())[0]
  return kota

In [12]:
# Apply fungsi
data['Tujuan'] = data['buyeraddress'].apply(lambda x : City(x))
data['Awal'] = data['selleraddress'].apply(lambda x : City(x))
data.tail()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,Tujuan,Awal
3176308,31504087640510,2020-04-02,2020-04-04,1970-01-01,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...,metro manila,metro manila
3176309,31504147352227,2020-04-03,2020-04-06,2020-04-07,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10...",metro manila,metro manila
3176310,31504462290482,2020-04-02,2020-04-04,1970-01-01,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ...",metro manila,metro manila
3176311,31504851495943,2020-04-03,2020-04-07,1970-01-01,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...,luzon,metro manila
3176312,31505199116340,2020-04-02,2020-04-06,2020-04-07,b4l13 TOLDA 792-A-5-A east...near CARRASCO Ang...,(Jeepney Kaligayan VANVAN Salud ASA Lauan A-55...,luzon,metro manila


In [17]:
def Expected(awal, tujuan):
  """
  Fungsi untuk menentukan expektasi hari pengiriman diantara 2 kota 
  barang berdasarkan perjanjian dengan pihak kurir.
  """
  try:
    return direc[awal][tujuan]  
  except:
    return direc[tujuan][awal]

In [25]:
# Apply fungsi
data['Expected'] = data[['Awal', 'Tujuan']].apply(lambda x : Expected(x['Awal'], x['Tujuan']), axis = 1)
data[['Awal', 'Tujuan', 'Expected']].head()

,Awal,Tujuan,Expected
0,metro manila,metro manila,3
1,metro manila,metro manila,3
2,metro manila,metro manila,3
3,metro manila,metro manila,3
4,metro manila,luzon,5


In [20]:
def Checker(start, end):
  """
  Fungsi Menghitung banyaknya hari kerja pada selang 2 tanggal
  dengan mengabaikan beberapa hari libur.
  """
  start, end = start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')
  libur = ['2020-03-08', '2020-03-25', '2020-03-30', '2020-03-31']
  count = np.busday_count(start, end, weekmask = [1,1,1,1,1,1,0], holidays = libur)
  return count

In [21]:
# Meenginisialisasi List untuk variable Is_Late
res = []

# Looping through
for i in tqdm(range(len(data['buyeraddress']))):
  # Menghitung banyak hari pengiriman pertama dan kedua
  count, second = Checker(data['pick'][i], data['1st_deliver_attempt'][i]), False
  if data['1st_deliver_attempt'][i] < data['2nd_deliver_attempt'][i]:
      second = Checker(data['1st_deliver_attempt'][i], data['2nd_deliver_attempt'][i])

  # Membandingkan Expected dengan banyak hari
  if count <= data['Expected'][i]:                                           
      if second:
          if second <= 3:
              res.append(0)
          else:
              res.append(1)
      else:
          res.append(0)
  else:
      res.append(1)

100%|██████████| 3176313/3176313 [07:10<00:00, 7378.18it/s]


## Mebuat Dataframe Sesuai Ketentuan & Export ke CSV

In [22]:
# Membuat Dataframe
submission = pd.DataFrame({'orderid' : data['orderid'].tolist(), 'is_late' : res})
submission.head(10)

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
5,2222597288,0
6,2222738456,0
7,2224695304,1
8,2224704587,0
9,2225138267,0


In [23]:
# Export
submission.to_csv('submission_1.csv', index = False)